In [1]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from numpy import sqrt

from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

# Load data

In [ ]:
from pathlib import Path
path = Path(__file__).parent.parent

In [2]:
# load dữ liệu từ file ratings train và test

train = pd.read_csv(path + "/training_data.csv")
test = pd.read_csv(path + "/testing_data.csv")

food_information = pd.read_excel(path + "/Food_description.xlsx")

In [3]:
food_information.head(3)

,FoodID,Tên món ăn,Vùng miền,Thành phần,"Thể loại (nướng, nước, khô, ...)",mô tả ngắn gọn ( <100 từ),Unnamed: 6
0,F1,Xôi,"Miền Bắc, Miền Trung, Miền Nam","Gạo nếp, đậu xanh, Vừng, Đậu phộng",Món khô,Xôi được làm từ gạo nếp hấp chín bằng hơi nước...,NaN
1,F2,Mì quảng,Miền Trung,"mì,nước lèo, thịt, tôm, cá, trứng , đậu phộng...",Món nước,"Có nguồn gốc ở Quảng Nam, mì quảng là món mì s...",NaN
2,F3,Phở bò,Miền Bắc,"bánh phở, nước lèo, thịt bò, trứng chần, hành ...",Món nước,"Món ăn nổi tiếng, được nhiều người ưu thích. Ă...",NaN


In [4]:
from scipy.sparse import csr_matrix
df_food = train.pivot(
    index = 'UserID',
    columns = 'FoodId',
    values = 'rating'
).fillna(0)

df_food

FoodId,F1,F10,F11,F12,F13,F14,F15,F16,F17,F18,...,F28,F29,F3,F30,F4,F5,F6,F7,F8,F9
UserID,,,,,,,,,,,,,,,,,,,,,
U1,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,5.0,0.0
U10,0.0,0.0,4.0,3.0,0.0,0.0,4.0,4.0,0.0,3.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0
U100,0.0,5.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0
U101,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,2.0,...,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
U102,0.0,2.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,4.0,0.0,2.0,2.0,5.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U95,0.0,0.0,5.0,3.0,0.0,0.0,4.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0
U96,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0
U97,5.0,5.0,0.0,5.0,0.0,3.0,0.0,0.0,3.0,5.0,...,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0


# Một vài hàm lấy dữ liệu

In [6]:
def get_rating(userid, foodid):
    return (train.loc[((train.UserID== userid)&(train.FoodId== foodid)), 'rating'].iloc[0])
#------------------------
def get_foodid(userid): 
    return (train.loc[(train.UserID==userid), 'FoodId'].tolist())
#----------------------
def get_userid(foodid): 
    return (train.loc[(train.FoodId==foodid), 'UserID'].tolist())
#------------------------
def get_food_title(foodid):
    return (food_information.loc[(food_information.FoodID==foodid), 'Tên món ăn'].iloc[0])
#------------------------
def get_average_rating(userid):
    rating_user = train.loc[((train.UserID== userid)), 'rating']
    return np.mean(rating_user)
#-----------------------
def get_average_rating_food(foodid):
    rating_food = train.loc[((train.FoodId== foodid)), 'rating']
    return np.mean(rating_food)

In [7]:
average_rating_userid = {}
for i in train["UserID"].unique():
    average_rating_userid[i]=get_average_rating(i)

In [8]:
average_rating_userid.items()

dict_items([('U112', 4.0), ('U130', 3.3333333333333335), ('U120', 3.111111111111111), ('U170', 3.1), ('U92', 3.375), ('U56', 5.0), ('U20', 3.6), ('U25', 3.1), ('U184', 3.5555555555555554), ('U129', 3.8), ('U1', 3.0), ('U94', 3.9), ('U57', 3.5714285714285716), ('U102', 3.0), ('U76', 3.4), ('U119', 3.5), ('U152', 4.0), ('U162', 3.111111111111111), ('U153', 3.4), ('U156', 4.0), ('U176', 3.4), ('U110', 4.0), ('U163', 4.2), ('U136', 3.7), ('U125', 4.4), ('U4', 3.5), ('U131', 3.3333333333333335), ('U166', 3.888888888888889), ('U15', 4.8), ('U142', 3.3), ('U52', 3.6), ('U231', 3.4), ('U23', 3.4), ('U49', 3.7), ('U164', 3.7), ('U221', 3.0), ('U202', 5.0), ('U67', 3.8), ('U207', 3.7), ('U177', 3.375), ('U200', 3.9), ('U109', 4.4), ('U160', 3.5), ('U91', 3.9), ('U101', 3.3), ('U39', 5.0), ('U65', 3.7), ('U40', 3.75), ('U3', 3.2), ('U29', 3.888888888888889), ('U124', 3.3333333333333335), ('U62', 3.7777777777777777), ('U46', 2.6666666666666665), ('U143', 3.5555555555555554), ('U14', 3.444444444444

# Các hàm tính độ tương tự 

## Độ đo Cosine

In [9]:
def cosine_similarity_item(food1, food2):
    both_watch_count = 0
    for userid in train.loc[train.FoodId == food1, 'UserID'].to_list():
        for userid in train.loc[train.FoodId == food2, 'UserID'].tolist():
            both_watch_count +=1
    if both_watch_count == 0:
        return 0

    rating1 = []
    rating2 = []

    for userid in train.loc[train.FoodId == food1, 'UserID'].to_list():
        if userid in train.loc[train.FoodId == food2, 'UserID'].tolist():
            rating1.append(get_rating(userid, food1))
            rating2.append(get_rating(userid, food2))

    if (norm(rating1)*norm(rating2)) == 0:
        return 0
    return dot(rating1, rating2)/(norm(rating1)*norm(rating2))

In [10]:
cosine_similarity_item("F1", "F10")

0.9668074931897049

## Độ đo tương tự Pearson

In [11]:
def pearson_correlation_item(food1, food2):
    both_watch_count = []
    for userid in train.loc[(train.FoodId == food1), 'UserID'].tolist():
        if userid in train.loc[(train.FoodId == food2), 'UserID'].tolist():
            both_watch_count.append(userid)
    if len(both_watch_count) == 0:
        return 0
    rating_sum_1 = sum([get_rating(userid, food1) for userid in both_watch_count])
    avg_rating_sum_1 = rating_sum_1/len(both_watch_count)

    rating_sum_2 = sum([get_rating(userid, food2) for userid in both_watch_count])
    avg_rating_sum_2 = rating_sum_2/len(both_watch_count)

    #Gia tri tren tu
    numerator = sum([(get_rating(userid, food1)- avg_rating_sum_1)*(get_rating(userid, food2) - avg_rating_sum_2) for userid in both_watch_count])
    denominator = sqrt(sum([pow((get_rating(userid, food1) - avg_rating_sum_1), 2) for userid in both_watch_count]))*sqrt(sum([pow((get_rating(userid, food2) - avg_rating_sum_2), 2) for userid in both_watch_count]))
    if denominator == 0:
        return 0
    return numerator/denominator

In [12]:
pearson_correlation_item("F1", "F7")

0.09688200325771523

# Hàm tính top k món ăn có độ tương đồng 

In [13]:
def get_similarity_item(food1, number_of_food, metric_similar):
    food_ids = train.FoodId.unique().tolist()

    similarity_score = []
    for foodID in food_ids:
        if (foodID != food1):
            if (metric_similar == "cosine"):
                similarity_score.append((cosine_similarity_item(food1, foodID), foodID))
            elif (metric_similar == "pearson"):
                similarity_score.append((pearson_correlation_item(food1, foodID), foodID))
    similarity_score.sort()
    similarity_score.reverse()
    return similarity_score[:number_of_food]

In [14]:
get_similarity_item("F10", 10, "pearson")

[(0.5466013526519906, 'F13'),
 (0.40446703212056356, 'F1'),
 (0.2694897223241833, 'F14'),
 (0.26444369362040315, 'F12'),
 (0.2518518518518519, 'F21'),
 (0.21146551050154147, 'F19'),
 (0.16931613528531078, 'F28'),
 (0.16217710142169822, 'F27'),
 (0.10239842231794327, 'F11'),
 (0.0569646690369124, 'F18')]

# Hàm khuyến nghị món ăn

## Khuyến nghị dựa trên trung bình đánh giá

In [15]:
def get_recommendation_tbdg_item(foodid, number_of_food, metric_similar):
    food_ids = train.FoodId.unique().tolist()
    total = {}
    number_rating = {}
    List_similar = get_similarity_item(foodid, number_of_food, metric_similar)
    for peason, food in List_similar:
        for userid in get_userid(food):
            if userid not in get_userid(foodid) or get_rating(userid, foodid) == 0:
                if userid not in total:
                    total[userid] = 0
                    number_rating[userid] = 0
                    total[userid] += get_rating(userid, food)
                    if get_rating(userid, food) != 0:
                        number_rating[userid] +=1
                else:
                    total[userid] += get_rating(userid, food)
                    if get_rating(userid, food) != 0:
                        number_rating[userid] +=1
                
    ranking = [(to/number_rating[userid], userid) for userid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(userid, score) for score, userid in ranking]

    return recomendations

## Khuyến nghị dựa theo trung bình đánh giá có trọng số

In [16]:
def get_recommendation_tbdgcts_item(foodid, number_of_food, metric_similar):
    food_ids = train.FoodId.unique().tolist()
    total = {}
    similarity_sum = {}
    List_similar = get_similarity_item(foodid, number_of_food, metric_similar)
    for peason, food in List_similar:
        score = peason
        for userid in get_userid(food):
            if userid not in get_userid(foodid) or get_rating(userid, foodid) == 0:
                if userid not in total:
                    total[userid] = 0
                    total[userid] += get_rating(userid, food)*score
                    similarity_sum [userid] = 0
                    similarity_sum [userid] +=score
                else:
                    total[userid] += get_rating(userid, food)*score
                    similarity_sum [userid] +=score
                
    ranking = [(to/similarity_sum[userid], userid) for userid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(userid, score) for score, userid in ranking]

    return recomendations

## Khuyến nghị dựa trên khoảng đánh giá

In [17]:
def get_recommendation_kdg_item(foodid, number_of_food, metric_similar):
    total = {}
    similarity_sum = {}
    List_similar = get_similarity_item(foodid, number_of_food, metric_similar)
    for peason, food in List_similar:
        score = peason
        for userid in get_userid(food):
            if userid not in get_userid(foodid) or get_rating(userid, foodid) == 0:
                if userid not in total:
                    total[userid] = 0
                    total[userid] += (get_rating(userid, food) - get_average_rating_food(food))*score
                    similarity_sum [userid] = 0
                    similarity_sum [userid] +=score
                else:
                    total[userid] += (get_rating(userid, food) - get_average_rating_food(food))*score
                    similarity_sum [userid] +=score
                
    ranking = [((get_average_rating_food(foodid) + to/similarity_sum[userid]), userid) for userid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(userid, score) for score, userid in ranking]

    return recomendations

## Khuyến nghị món ăn

In [18]:
def get_recommendation_item(userid, number_of_food, metrics_rating, metric_similar):
    if userid not in list(train["FoodId"].unique()):
        ranking = [(to, userid) for userid, to in average_rating_userid.items()]
        ranking.sort()
        ranking.reverse()
        recomendations = [(userid, score) for score, userid in ranking]
        return recomendations
    if (metrics_rating == "tbdg"):
        recomendations = get_recommendation_tbdg_item(userid, number_of_food, metric_similar)
        return recomendations
    elif (metrics_rating == "tbdgcts"):
        recomendations = get_recommendation_tbdgcts_item(userid, number_of_food, metric_similar)
        return recomendations
    elif (metrics_rating == "kdg"):
        recomendations = get_recommendation_kdg_item(userid, number_of_food, metric_similar)
        return recomendations

In [19]:
food = "F7"
number_of_food = 10
metrics_rating = "kdg" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "cosine" # 1 trong 2 ["cosine", "peason"]
food_recommend = get_recommendation_item(food, number_of_food, metrics_rating, metrics_similar)

# Đánh giá phương pháp

In [20]:
def get_recommendation_4test_item(X_test, number_of_food, metrics_rating, metric_similar):
    X_test_pred = []
    X_test_userid = X_test['UserID'].tolist()
    X_test_FoodId = X_test['FoodId'].tolist()
    print("Dự đoán cho tập test với {} điểm dữ liệu".format(len(X_test_FoodId)))
    for i in range(len(X_test_FoodId)):
        list_R = get_recommendation_item(X_test_FoodId[i], number_of_food, metrics_rating, metrics_similar)
        check = 0
        for j in list_R:
            if(X_test_userid[i] == j[0]):
                X_test_pred.append(j[1])
                check = 1
        if(check == 0 ):
            X_test_pred.append(0)
        print('----------dự đoán cho dòng thứ {}'.format(i), 'là {}'.format(X_test_pred[i]))
    return X_test_pred

## Dự đoán bằng pp TBDGCTS và cosine

In [21]:
import time
from sklearn.metrics import f1_score, precision_score, recall_score

In [22]:
start = time.time()
number_similar = 10
metrics_rating = "tbdgcts" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "cosine" # 1 trong 2 ["cosine", "peason"]
X_test_pred_rating = get_recommendation_4test_item(test, number_similar, metrics_rating, metrics_similar)

end = time.time()

print("Tổng thời gian dự đoán :", end - start)

Dự đoán cho tập test với 117 điểm dữ liệu
----------dự đoán cho dòng thứ 0 là 3.75051640238175


KeyboardInterrupt: 

In [ ]:
X_test_pred_rating_select = []
X_test_true_rating_select = []

for i in range(len(X_test_pred_rating)):
    if (X_test_pred_rating[i] != 0):
        X_test_pred_rating_select.append(X_test_pred_rating[i])
        X_test_true_rating_select.append(test['rating'].tolist()[i])
        X_test_pred_rating_select_round = [round(num) for num in X_test_pred_rating_select]

mse = mean_squared_error(X_test_true_rating_select, X_test_pred_rating_select)
rmse = math.sqrt(mse)
mae = mean_absolute_error(X_test_true_rating_select, X_test_pred_rating_select)

precision = precision_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
recall = recall_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
f1 = f1_score(X_test_true_rating_select, X_test_pred_rating_select_round, average='macro')
#X_test_true_rating_select.sort(reverse = True)
#max = X_test_true_rating_select[0]
#min = X_test_true_rating_select[len(X_test_true_rating_select)-1]
#nmae = mae/(max-min)
print('---------------------------------')
#print('Đánh giá trên độ đo MSE:', mse)
print('Đánh giá trên độ đo RMSE:', rmse)
print('Đánh giá trên độ đo MAE:', mae)
#print('Đánh giá trên độ đo NMAE:', nmae)
print('Đánh giá trên độ đo precision:', precision)
print('Đánh giá trên độ đo recall:', recall)
print('Đánh giá trên độ đo f1_score:', f1)

---------------------------------
Đánh giá trên độ đo RMSE: 1.169087375075723
Đánh giá trên độ đo MAE: 0.9242503012945611
Đánh giá trên độ đo precision: 0.2480433033428425
Đánh giá trên độ đo recall: 0.23860434152425253
Đánh giá trên độ đo f1_score: 0.23363095238095238


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Dự đoán và đánh giá bằng pp TBDGCTS và peason

In [ ]:
import time
start = time.time()
number_similar = 10
metrics_rating = "tbdgcts" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "pearson" # 1 trong 2 ["cosine", "peason"]
X_test_pred_rating = get_recommendation_4test_item(test, number_similar, metrics_rating, metrics_similar)

end = time.time()

print("Tổng thời gian dự đoán :", end - start)

Dự đoán cho tập test với 117 điểm dữ liệu
----------dự đoán cho dòng thứ 0 là 4.0
----------dự đoán cho dòng thứ 1 là 3.2174705934343533
----------dự đoán cho dòng thứ 2 là 3.3225793067815466
----------dự đoán cho dòng thứ 3 là 5.0
----------dự đoán cho dòng thứ 4 là 3.799809101196309
----------dự đoán cho dòng thứ 5 là 4.629592579364961
----------dự đoán cho dòng thứ 6 là 4.0
----------dự đoán cho dòng thứ 7 là 3.312777556663013
----------dự đoán cho dòng thứ 8 là 5.0
----------dự đoán cho dòng thứ 9 là 4.6967252741947405
----------dự đoán cho dòng thứ 10 là 2.400255951573887
----------dự đoán cho dòng thứ 11 là 4.0
----------dự đoán cho dòng thứ 12 là 2.7568583595001357
----------dự đoán cho dòng thứ 13 là 4.440789257469897
----------dự đoán cho dòng thứ 14 là 2.694942381883323
----------dự đoán cho dòng thứ 15 là 3.6832505062015493
----------dự đoán cho dòng thứ 16 là 2.5632036342779627
----------dự đoán cho dòng thứ 17 là 4.107379305310181
----------dự đoán cho dòng thứ 18 là 3.155

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


----------dự đoán cho dòng thứ 53 là 4.553485173785192
----------dự đoán cho dòng thứ 54 là 3.000000000000001
----------dự đoán cho dòng thứ 55 là 3.7381360284231566
----------dự đoán cho dòng thứ 56 là 3.059006788297257
----------dự đoán cho dòng thứ 57 là 3.6280330553192925
----------dự đoán cho dòng thứ 58 là 4.084648507015832
----------dự đoán cho dòng thứ 59 là 4.133235867286004
----------dự đoán cho dòng thứ 60 là 3.1209926003894655
----------dự đoán cho dòng thứ 61 là 3.2847420903227746
----------dự đoán cho dòng thứ 62 là 3.3610582363754045
----------dự đoán cho dòng thứ 63 là 3.5925804380921202
----------dự đoán cho dòng thứ 64 là 3.659917040361692
----------dự đoán cho dòng thứ 65 là 3.6463464285535934
----------dự đoán cho dòng thứ 66 là 4.1158252366772805
----------dự đoán cho dòng thứ 67 là 2.929661208293428
----------dự đoán cho dòng thứ 68 là 3.5396830435242164
----------dự đoán cho dòng thứ 69 là 2.830349706679435
----------dự đoán cho dòng thứ 70 là 3.573587883989774
-

In [ ]:
X_test_pred_rating = list(np.where(np.isnan(X_test_pred_rating), 0, X_test_pred_rating))

X_test_pred_rating_select = []
X_test_true_rating_select = []

for i in range(len(X_test_pred_rating)):
    if (X_test_pred_rating[i] != 0):
        X_test_pred_rating_select.append(X_test_pred_rating[i])
        X_test_true_rating_select.append(test['rating'].tolist()[i])
        X_test_pred_rating_select_round = [round(num) for num in X_test_pred_rating_select]

mse = mean_squared_error(X_test_true_rating_select, X_test_pred_rating_select)
rmse = math.sqrt(mse)
mae = mean_absolute_error(X_test_true_rating_select, X_test_pred_rating_select)

precision = precision_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
recall = recall_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
f1 = f1_score(X_test_true_rating_select, X_test_pred_rating_select_round, average='macro')
#X_test_true_rating_select.sort(reverse = True)
#max = X_test_true_rating_select[0]
#min = X_test_true_rating_select[len(X_test_true_rating_select)-1]
#nmae = mae/(max-min)
print('---------------------------------')
#print('Đánh giá trên độ đo MSE:', mse)
print('Đánh giá trên độ đo RMSE:', rmse)
print('Đánh giá trên độ đo MAE:', mae)
#print('Đánh giá trên độ đo NMAE:', nmae)
print('Đánh giá trên độ đo precision:', precision)
print('Đánh giá trên độ đo recall:', recall)
print('Đánh giá trên độ đo f1_score:', f1)

---------------------------------
Đánh giá trên độ đo RMSE: 1.003392851587606
Đánh giá trên độ đo MAE: 0.8102597358265085
Đánh giá trên độ đo precision: 0.3780487804878049
Đánh giá trên độ đo recall: 0.3064860276235488
Đánh giá trên độ đo f1_score: 0.3008975352555822


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Dự đoán và đánh giá bằng pp TBDG và cosine

In [ ]:
import time
start = time.time()
user = "U1"
number_similar = 10
metrics_rating = "tbdg" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "cosine" # 1 trong 2 ["cosine", "peason"]
X_test_pred_rating = get_recommendation_4test_item(test, number_similar, metrics_rating, metrics_similar)

end = time.time()

print("Tổng thời gian dự đoán :", end - start)

Dự đoán cho tập test với 117 điểm dữ liệu
----------dự đoán cho dòng thứ 0 là 0
----------dự đoán cho dòng thứ 1 là 3.5
----------dự đoán cho dòng thứ 2 là 3.3333333333333335
----------dự đoán cho dòng thứ 3 là 5.0
----------dự đoán cho dòng thứ 4 là 3.75
----------dự đoán cho dòng thứ 5 là 4.5
----------dự đoán cho dòng thứ 6 là 4.0
----------dự đoán cho dòng thứ 7 là 3.75
----------dự đoán cho dòng thứ 8 là 5.0
----------dự đoán cho dòng thứ 9 là 5.0
----------dự đoán cho dòng thứ 10 là 2.8
----------dự đoán cho dòng thứ 11 là 0
----------dự đoán cho dòng thứ 12 là 2.6666666666666665
----------dự đoán cho dòng thứ 13 là 4.5
----------dự đoán cho dòng thứ 14 là 2.0
----------dự đoán cho dòng thứ 15 là 4.0
----------dự đoán cho dòng thứ 16 là 3.0
----------dự đoán cho dòng thứ 17 là 4.0
----------dự đoán cho dòng thứ 18 là 3.0
----------dự đoán cho dòng thứ 19 là 0
----------dự đoán cho dòng thứ 20 là 3.0
----------dự đoán cho dòng thứ 21 là 0
----------dự đoán cho dòng thứ 22 là 4.5
-

In [ ]:
X_test_pred_rating = list(np.where(np.isnan(X_test_pred_rating), 0, X_test_pred_rating))

X_test_pred_rating_select = []
X_test_true_rating_select = []

for i in range(len(X_test_pred_rating)):
    if (X_test_pred_rating[i] != 0):
        X_test_pred_rating_select.append(X_test_pred_rating[i])
        X_test_true_rating_select.append(test['rating'].tolist()[i])
        X_test_pred_rating_select_round = [round(num) for num in X_test_pred_rating_select]

mse = mean_squared_error(X_test_true_rating_select, X_test_pred_rating_select)
rmse = math.sqrt(mse)
mae = mean_absolute_error(X_test_true_rating_select, X_test_pred_rating_select)

precision = precision_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
recall = recall_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
f1 = f1_score(X_test_true_rating_select, X_test_pred_rating_select_round, average='macro')
#X_test_true_rating_select.sort(reverse = True)
#max = X_test_true_rating_select[0]
#min = X_test_true_rating_select[len(X_test_true_rating_select)-1]
#nmae = mae/(max-min)
print('---------------------------------')
#print('Đánh giá trên độ đo MSE:', mse)
print('Đánh giá trên độ đo RMSE:', rmse)
print('Đánh giá trên độ đo MAE:', mae)
#print('Đánh giá trên độ đo NMAE:', nmae)
print('Đánh giá trên độ đo precision:', precision)
print('Đánh giá trên độ đo recall:', recall)
print('Đánh giá trên độ đo f1_score:', f1)

---------------------------------
Đánh giá trên độ đo RMSE: 1.1689869326549158
Đánh giá trên độ đo MAE: 0.924198717948718
Đánh giá trên độ đo precision: 0.24376623376623377
Đánh giá trên độ đo recall: 0.23259766744193885
Đánh giá trên độ đo f1_score: 0.22302367941712203


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Dự đoán và đánh giá bằng pp TBDG và peason


In [ ]:
import time
start = time.time()
number_similar = 10
metrics_rating = "tbdg" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "pearson" # 1 trong 2 ["cosine", "pearson"]
X_test_pred_rating = get_recommendation_4test_item(test, number_similar, metrics_rating, metrics_similar)

end = time.time()

print("Tổng thời gian dự đoán :", end - start)

Dự đoán cho tập test với 117 điểm dữ liệu
----------dự đoán cho dòng thứ 0 là 4.0
----------dự đoán cho dòng thứ 1 là 3.3333333333333335
----------dự đoán cho dòng thứ 2 là 3.3333333333333335
----------dự đoán cho dòng thứ 3 là 5.0
----------dự đoán cho dòng thứ 4 là 4.285714285714286
----------dự đoán cho dòng thứ 5 là 4.666666666666667
----------dự đoán cho dòng thứ 6 là 4.0
----------dự đoán cho dòng thứ 7 là 3.3333333333333335
----------dự đoán cho dòng thứ 8 là 5.0
----------dự đoán cho dòng thứ 9 là 4.6
----------dự đoán cho dòng thứ 10 là 2.6
----------dự đoán cho dòng thứ 11 là 4.0
----------dự đoán cho dòng thứ 12 là 3.0
----------dự đoán cho dòng thứ 13 là 4.333333333333333
----------dự đoán cho dòng thứ 14 là 3.0
----------dự đoán cho dòng thứ 15 là 4.0
----------dự đoán cho dòng thứ 16 là 2.6666666666666665
----------dự đoán cho dòng thứ 17 là 4.142857142857143
----------dự đoán cho dòng thứ 18 là 3.4285714285714284
----------dự đoán cho dòng thứ 19 là 4.0
----------dự đoán

In [ ]:
X_test_pred_rating = list(np.where(np.isnan(X_test_pred_rating), 0, X_test_pred_rating))

X_test_pred_rating_select = []
X_test_true_rating_select = []

for i in range(len(X_test_pred_rating)):
    if (X_test_pred_rating[i] != 0):
        X_test_pred_rating_select.append(X_test_pred_rating[i])
        X_test_true_rating_select.append(test['rating'].tolist()[i])
        X_test_pred_rating_select_round = [round(num) for num in X_test_pred_rating_select]

mse = mean_squared_error(X_test_true_rating_select, X_test_pred_rating_select)
rmse = math.sqrt(mse)
mae = mean_absolute_error(X_test_true_rating_select, X_test_pred_rating_select)

precision = precision_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
recall = recall_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
f1 = f1_score(X_test_true_rating_select, X_test_pred_rating_select_round, average='macro')
#X_test_true_rating_select.sort(reverse = True)
#max = X_test_true_rating_select[0]
#min = X_test_true_rating_select[len(X_test_true_rating_select)-1]
#nmae = mae/(max-min)
print('---------------------------------')
#print('Đánh giá trên độ đo MSE:', mse)
print('Đánh giá trên độ đo RMSE:', rmse)
print('Đánh giá trên độ đo MAE:', mae)
#print('Đánh giá trên độ đo NMAE:', nmae)
print('Đánh giá trên độ đo precision:', precision)
print('Đánh giá trên độ đo recall:', recall)
print('Đánh giá trên độ đo f1_score:', f1)

---------------------------------
Đánh giá trên độ đo RMSE: 0.9909952979323673
Đánh giá trên độ đo MAE: 0.8045991045991046
Đánh giá trên độ đo precision: 0.27662538699690403
Đánh giá trên độ đo recall: 0.2222456752168127
Đánh giá trên độ đo f1_score: 0.19974481658692184


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Dự đoán và đánh giá bằng pp KDG và cosine

In [ ]:
import time
start = time.time()
user = "U1"
number_similar = 10
metrics_rating = "kdg" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "cosine" # 1 trong 2 ["cosine", "peason"]
X_test_pred_rating = get_recommendation_4test_item(test, number_similar, metrics_rating, metrics_similar)

end = time.time()

print("Tổng thời gian dự đoán :", end - start)

Dự đoán cho tập test với 117 điểm dữ liệu
----------dự đoán cho dòng thứ 0 là 0
----------dự đoán cho dòng thứ 1 là 3.333828363578004
----------dự đoán cho dòng thứ 2 là 3.327110157052952
----------dự đoán cho dòng thứ 3 là 4.9636010182835975
----------dự đoán cho dòng thứ 4 là 3.36031808039795
----------dự đoán cho dòng thứ 5 là 4.463100098195268
----------dự đoán cho dòng thứ 6 là 3.8735166633680618
----------dự đoán cho dòng thứ 7 là 3.5840609736729006
----------dự đoán cho dòng thứ 8 là 4.9636010182835975
----------dự đoán cho dòng thứ 9 là 5.158049312946463
----------dự đoán cho dòng thứ 10 là 2.9424599354850467
----------dự đoán cho dòng thứ 11 là 0
----------dự đoán cho dòng thứ 12 là 2.8182322328715466
----------dự đoán cho dòng thứ 13 là 3.9787615754801338
----------dự đoán cho dòng thứ 14 là 1.710801393728223
----------dự đoán cho dòng thứ 15 là 4.481836571163448
----------dự đoán cho dòng thứ 16 là 3.418115466658739
----------dự đoán cho dòng thứ 17 là 4.082877286985849
----

In [ ]:
X_test_pred_rating = list(np.where(np.isnan(X_test_pred_rating), 0, X_test_pred_rating))

X_test_pred_rating_select = []
X_test_true_rating_select = []

for i in range(len(X_test_pred_rating)):
    if (X_test_pred_rating[i] != 0):
        X_test_pred_rating_select.append(X_test_pred_rating[i])
        X_test_true_rating_select.append(test['rating'].tolist()[i])
        X_test_pred_rating_select_round = [round(num) for num in X_test_pred_rating_select]

mse = mean_squared_error(X_test_true_rating_select, X_test_pred_rating_select)
rmse = math.sqrt(mse)
mae = mean_absolute_error(X_test_true_rating_select, X_test_pred_rating_select)

precision = precision_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
recall = recall_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
f1 = f1_score(X_test_true_rating_select, X_test_pred_rating_select_round, average='macro')
#X_test_true_rating_select.sort(reverse = True)
#max = X_test_true_rating_select[0]
#min = X_test_true_rating_select[len(X_test_true_rating_select)-1]
#nmae = mae/(max-min)
print('---------------------------------')
#print('Đánh giá trên độ đo MSE:', mse)
print('Đánh giá trên độ đo RMSE:', rmse)
print('Đánh giá trên độ đo MAE:', mae)
#print('Đánh giá trên độ đo NMAE:', nmae)
print('Đánh giá trên độ đo precision:', precision)
print('Đánh giá trên độ đo recall:', recall)
print('Đánh giá trên độ đo f1_score:', f1)

---------------------------------
Đánh giá trên độ đo RMSE: 1.0809289596374347
Đánh giá trên độ đo MAE: 0.8501236274416227
Đánh giá trên độ đo precision: 0.2787166537166537
Đánh giá trên độ đo recall: 0.2650595218031258
Đánh giá trên độ đo f1_score: 0.2584357926035395


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Dự đoán và đánh giá bằng pp KDG và peason

In [ ]:
import time
start = time.time()
user = "U1"
number_similar = 10
metrics_rating = "kdg" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "pearson" # 1 trong 2 ["cosine", "peason"]
X_test_pred_rating = get_recommendation_4test_item(test, number_similar, metrics_rating, metrics_similar)

end = time.time()

print("Tổng thời gian dự đoán :", end - start)

Dự đoán cho tập test với 117 điểm dữ liệu
----------dự đoán cho dòng thứ 0 là 3.6771550983447683
----------dự đoán cho dòng thứ 1 là 3.2934803379766837
----------dự đoán cho dòng thứ 2 là 3.368114283991328
----------dự đoán cho dòng thứ 3 là 5.03087913889123
----------dự đoán cho dòng thứ 4 là 3.6573732250783775
----------dự đoán cho dòng thứ 5 là 4.660471718256191
----------dự đoán cho dòng thứ 6 là 3.900140632257229
----------dự đoán cho dòng thứ 7 là 3.3887873012053427
----------dự đoán cho dòng thứ 8 là 4.965275174176311
----------dự đoán cho dòng thứ 9 là 4.767051685484217
----------dự đoán cho dòng thứ 10 là 2.6578916601957596
----------dự đoán cho dòng thứ 11 là 4.329175138529981
----------dự đoán cho dòng thứ 12 là 2.965796501747763
----------dự đoán cho dòng thứ 13 là 4.395971435389873
----------dự đoán cho dòng thứ 14 là 2.1815281039748564
----------dự đoán cho dòng thứ 15 là 4.230278790153998
----------dự đoán cho dòng thứ 16 là 3.120962471374612
----------dự đoán cho dòng t

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


----------dự đoán cho dòng thứ 53 là 4.984394360408602
----------dự đoán cho dòng thứ 54 là 3.3554700339867596
----------dự đoán cho dòng thứ 55 là 3.7836710056329386
----------dự đoán cho dòng thứ 56 là 3.0975209776248533
----------dự đoán cho dòng thứ 57 là 3.335214664202759
----------dự đoán cho dòng thứ 58 là 4.199161684986175
----------dự đoán cho dòng thứ 59 là 4.043417164613546
----------dự đoán cho dòng thứ 60 là 3.1970023449317946
----------dự đoán cho dòng thứ 61 là 4.0735927975376525
----------dự đoán cho dòng thứ 62 là 3.5099536550247303
----------dự đoán cho dòng thứ 63 là 3.44031346523018
----------dự đoán cho dòng thứ 64 là 3.7054520175714734
----------dự đoán cho dòng thứ 65 là 3.3616152139967794
----------dự đoán cho dòng thứ 66 là 4.1437818906069515
----------dự đoán cho dòng thứ 67 là 3.258836346823409
----------dự đoán cho dòng thứ 68 là 3.345800701790289
----------dự đoán cho dòng thứ 69 là 2.6274351036174894
----------dự đoán cho dòng thứ 70 là 3.421320911127834
-

In [ ]:
X_test_pred_rating = list(np.where(np.isnan(X_test_pred_rating), 0, X_test_pred_rating))

X_test_pred_rating_select = []
X_test_true_rating_select = []

for i in range(len(X_test_pred_rating)):
    if (X_test_pred_rating[i] != 0):
        X_test_pred_rating_select.append(X_test_pred_rating[i])
        X_test_true_rating_select.append(test['rating'].tolist()[i])
        X_test_pred_rating_select_round = [round(num) for num in X_test_pred_rating_select]

mse = mean_squared_error(X_test_true_rating_select, X_test_pred_rating_select)
rmse = math.sqrt(mse)
mae = mean_absolute_error(X_test_true_rating_select, X_test_pred_rating_select)

precision = precision_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
recall = recall_score(X_test_true_rating_select, X_test_pred_rating_select_round,average='macro')
f1 = f1_score(X_test_true_rating_select, X_test_pred_rating_select_round, average='macro')
#X_test_true_rating_select.sort(reverse = True)
#max = X_test_true_rating_select[0]
#min = X_test_true_rating_select[len(X_test_true_rating_select)-1]
#nmae = mae/(max-min)
print('---------------------------------')
#print('Đánh giá trên độ đo MSE:', mse)
print('Đánh giá trên độ đo RMSE:', rmse)
print('Đánh giá trên độ đo MAE:', mae)
#print('Đánh giá trên độ đo NMAE:', nmae)
print('Đánh giá trên độ đo precision:', precision)
print('Đánh giá trên độ đo recall:', recall)
print('Đánh giá trên độ đo f1_score:', f1)

---------------------------------
Đánh giá trên độ đo RMSE: 0.925214053911023
Đánh giá trên độ đo MAE: 0.746055875087061
Đánh giá trên độ đo precision: 0.32396004378763
Đánh giá trên độ đo recall: 0.2408093577448416
Đánh giá trên độ đo f1_score: 0.24838116894697956


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Đánh giá bằng MRR và NDCG

## Các hàm dùng chung

In [ ]:
GroundTruth = pd.read_csv(path + "/groundtruth.csv")

In [ ]:
def Get_rating_for_each_user(ListUser, number_of_food, metrics_rating, metric_similar):
    Recommend_for_ranking = []
    for foodid in list(train.FoodId.unique()):
        food_recommend = get_recommendation_item(foodid, number_of_food, metrics_rating, metric_similar)
        for item in food_recommend:
            if item[0] in ListUser:
                item_new = (foodid,) + item
                Recommend_for_ranking.append(item_new)
    return Recommend_for_ranking

In [ ]:
def Get_Predict_List(Recommend_list, userid, number_of_food, metrics_rating, metric_similar):
    MRR_predict = []
    predict = []
    for row in Recommend_list:
        if row[1] == userid:
            predict.append(row)
    if len(predict) > 10:
        top_N =10
    else:
        top_N = len(predict)
    predict = predict[:top_N]
    ground_truth = GroundTruth.loc[GroundTruth["UserID"] == userid].head(top_N)
    for i in range(len(ground_truth)):
        if predict[i][0] in list(ground_truth.FoodId):
            if (((GroundTruth.loc[GroundTruth["FoodId"] == predict[i][0]].rating.iloc[0]) >= 3) and (predict[i][2] >=3)):
                MRR_predict.append(1)
            else:
                MRR_predict.append(0)
        else :
            MRR_predict.append(0)
    return MRR_predict

## Hàm tính MRR

In [ ]:
def RRu_score(MRR_predict):
    RRu = 0
    for i in range(len(MRR_predict)):
        rr = MRR_predict[i]/(i+1)
        RRu+=rr
    return RRu

In [ ]:
def MRR(ListUser, number_of_user, metrics_rating, metric_similar):
    # ListUser: Danh sách các user GroundTruth
    # Recommend_List: Danh sách các món được predict cho GroundTruth
    # number_of_user: Số user tương đồng
    # metrics_rating: Phương pháp đánh giá
    # Metric_similar : "Độ đo cosine"
    # Top N: Số lương Top Danh sách muốn tính cho MRR
    # Kết quả trả về là kết quả MRR cho pp tương ứng
    MRR = 0
    Recommend_list = Get_rating_for_each_user(ListUser, number_of_food, metrics_rating, metric_similar)
    for user in ListUser:
        MRR_predict = Get_Predict_List(Recommend_list, user, number_of_user, metrics_rating, metric_similar)
        RRu = RRu_score(MRR_predict)
        MRR += RRu
        print("MRRu score:", RRu)
    return (1/len(ListUser))*MRR

In [ ]:
ListUser = ["U236", "U237", "U238", "U239", "U240"]
number_of_food = 10
metrics_rating = "kdg"
metric_similar = "cosine"
MRR_score = MRR(ListUser, number_of_food, metrics_rating, metric_similar)

MRRu score: 0.25
MRRu score: 0.26785714285714285
MRRu score: 1.7845238095238096
MRRu score: 2.141666666666667
MRRu score: 1.2


In [ ]:
print(MRR_score)

1.128809523809524


In [ ]:
ListUser = ["U236", "U237", "U238", "U239", "U240"]
number_of_food = 10
metrics_rating = "kdg"
metric_similar = "pearson"
MRR_score = MRR(ListUser, number_of_food, metrics_rating, metric_similar)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


MRRu score: 0.25
MRRu score: 0.325
MRRu score: 1.7845238095238096
MRRu score: 2.041666666666667
MRRu score: 0.2


In [ ]:
print(MRR_score)

0.9202380952380955


## Hàm tính NDCG

In [ ]:
from sklearn.metrics import ndcg_score
import math

In [ ]:
def DCG_score(predict_ranking):
    DCG = 0
    for i in range(len(predict_ranking)):
        DCG += predict_ranking[i]/math.log2(i+1+1)
    return DCG

In [ ]:
def iDCG_score(true_ranking):
    DCG = 0
    for i in range(len(true_ranking)):
        DCG += true_ranking[i]/math.log2(i+1+1)
    return DCG

In [ ]:
def NDCG_for_listUser(ListUser, number_of_user, metrics_rating, metric_similar):
    ndcg = 0
    for user in ListUser:
        Recommend_list = Get_rating_for_each_user(ListUser, number_of_food, metrics_rating, metric_similar)
        prec_ranking = Get_Predict_List(Recommend_list, user, number_of_user, metrics_rating, metric_similar)
        true_ranking = sorted(prec_ranking, reverse=True)
        print("predict_ranking")
        print(prec_ranking)
        print("true_ranking")
        print(true_ranking)
        dcg = DCG_score(prec_ranking)
        idcg = iDCG_score(true_ranking)
        ndcg += dcg/idcg
        print("ndcg:", dcg/idcg)
    return (1/len(ListUser))*ndcg

In [ ]:
ListUser = ["U236", "U237", "U238", "U239", "U240"]
ndcg_score = NDCG_for_listUser(ListUser, 10, "kdg", "cosine")
print("NDCG_KDG_cosine", ndcg_score)

predict_ranking
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
true_ranking
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ndcg: 0.43067655807339306
predict_ranking
[0, 0, 0, 0, 0, 0, 1, 1, 0, 0]
true_ranking
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
ndcg: 0.39780880120575696
predict_ranking
[1, 0, 0, 1, 0, 1, 1, 1, 0, 1]
true_ranking
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
ndcg: 0.8245149522908417
predict_ranking
[1, 1, 0, 1, 0, 1, 0, 1, 0, 1]
true_ranking
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
ndcg: 0.9145683473035053
predict_ranking
[1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
true_ranking
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
ndcg: 0.8503449055347546
NDCG_KDG_cosine 0.6835827128816503


In [ ]:
ListUser = ["U236", "U237", "U238", "U239", "U240"]
ndcg_score = NDCG_for_listUser(ListUser, 10, "kdg", "pearson")
print("NDCG_KDG_pearson", ndcg_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


predict_ranking
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
true_ranking
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ndcg: 0.43067655807339306
predict_ranking
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
true_ranking
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
ndcg: 0.430624116386567
predict_ranking
[1, 0, 0, 1, 0, 1, 1, 1, 0, 1]
true_ranking
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
ndcg: 0.8245149522908417
predict_ranking
[1, 1, 0, 1, 0, 1, 0, 1, 0, 0]
true_ranking
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
ndcg: 0.9270192549176082
predict_ranking
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
true_ranking
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ndcg: 0.38685280723454163
NDCG_KDG_pearson 0.5999375377805903
